This notebook contains the implementation of SVM to Pneumonia data set using SVM with and without PCA 

In [39]:
import os
import numpy as np
import pydicom
import cv2
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm


In [ ]:
import cv2

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm


/Users/Lpinheiro/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:

# setting paths 
data_dir = '/Users/Lpinheiro/Desktop/MS Data Science/DSP 577/rsna-pneumonia-detection-challenge/pneumonia_data'


In [5]:
train_images_dir = os.path.join(data_dir, 'stage_2_train_images')

In [7]:

train_labels_file = os.path.join(data_dir, 'stage_2_train_labels.csv')


In [8]:

# loading labels 
import pandas as pd
labels = pd.read_csv(train_labels_file)


In [9]:

# labels: 1 = pneumonia, 0 = no pneumonia
labels['Target'] = labels['Target'].apply(lambda x: 1 if x == 1 else 0)
labels = labels[['patientId', 'Target']].drop_duplicates()


In [10]:
# image preprocessing 

def preprocess_images(image_dir, labels_df, img_size):
    images = []
    targets = []
    for _, row in tqdm(labels_df.iterrows(), total=len(labels_df)):
        patient_id = row['patientId']
        target = row['Target']
        image_path = os.path.join(image_dir, f'{patient_id}.dcm')

        # Load and preprocess DICOM image
        try:
            dicom = pydicom.dcmread(image_path)
            img = dicom.pixel_array
            img_resized = cv2.resize(img, (img_size, img_size))
            images.append(img_resized.flatten())  # Flatten image to 1D
            targets.append(target)
        except Exception as e:
            print(f'Error processing {patient_id}: {e}')

    return np.array(images), np.array(targets)


In [11]:

# PCA
from sklearn.decomposition import PCA

def apply_pca(images, n_components):
    pca = PCA(n_components=n_components)
    reduced_images = pca.fit_transform(images)
    print(f'Dimensionality reduced to {n_components} components.')
    return reduced_images, pca


In [40]:
# t-SNE

from sklearn.manifold import TSNE

def apply_tsne(images, n_components, perplexity=30, random_state=None):
    tsne = TSNE(n_components=n_components, perplexity=perplexity, random_state=random_state)
    reduced_images = tsne.fit_transform(images)
    print(f'Dimensionality reduced to {n_components} components using t-SNE.')
    return reduced_images



In [17]:
# Preprocess images
X, y = preprocess_images(image_dir=train_images_dir, labels_df=labels, img_size=64)


100%|██████████| 26684/26684 [05:40<00:00, 78.36it/s]


In [38]:

# Apply PCA
X_pca, pca_model = apply_pca(X, n_components=50)


Dimensionality reduced to 50 components.


In [78]:

# Apply t-SNE
X_tsne = apply_tsne(X, n_components=3, perplexity=30, random_state=31415)


Dimensionality reduced to 3 components using t-SNE.


In [79]:

from sklearn.model_selection import train_test_split



In [86]:

# select a subset of 1000 images
X_subset, _, y_subset, _ = train_test_split(X_tsne, y, train_size=1000, random_state=42, stratify=y)


In [87]:

# raining and testing splits
X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.2, random_state=31415, stratify=y_subset)


In [88]:

# scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [89]:

# train SVM
svm_model = SVC(kernel='linear', C=1.0, random_state=31415)
svm_model.fit(X_train_scaled, y_train)


SVC(kernel='linear', random_state=31415)

In [ ]:

# evaluate
y_pred = svm_model.predict(X_test_scaled)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:')
print(classification_report(y_test, y_pred))


Accuracy: 0.775
Classification Report:
              precision    recall  f1-score   support

           0       0.78      1.00      0.87       155
           1       0.00      0.00      0.00        45

    accuracy                           0.78       200
   macro avg       0.39      0.50      0.44       200
weighted avg       0.60      0.78      0.68       200



/Users/Lpinheiro/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/Lpinheiro/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/Lpinheiro/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
